### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__Mount Everest Climbing Deaths__  
  URL:  
  ・https://www.kaggle.com/shivamb/mount-everest-climbing-deaths  
  DATA:  
  ・mount_everest_deaths.csv

In [ ]:
%matplotlib inline

import branca.colormap as cm
from decimal import Decimal, ROUND_HALF_UP
import folium
from geopy.distance import geodesic
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns

### 1. データ型の変換

エベレスト登山死者データをロード。

中身を確認。

どんなデータがありますか？

データの型と欠損を確認。

#### 1-1. 日時型への変換

まずは日付の変換をします。

`Date`は日時型ではないようですので日付のフォーマットを指定して日時型に変換し、新たな列の`death_date`に格納します。

参考）  
strftime() と strptime() の振る舞い  
https://docs.python.org/ja/3/library/datetime.html#strftime-and-strptime-behavior

変換失敗がないか確認。

失敗が１件あるようなので、どんなデータで失敗しているかを確認。

これはデータの問題なので、`death_date`を直接設定してしまいます。

もう一度変換失敗がないか確認。

死亡年をX軸、死亡者数をY軸とした折れ線グラフを描いてみましょう。

死亡年でグループ分けし、インデックスが`death_year`、列が`死亡者数`のデータフレームを作成します。

`death_year`の値が飛び飛びなので、欠けている年をすべて埋めることとします。

まずはインデックスを日付型（各年の元旦）に変換します。

インデックスを日付型に変換したら、今度は`pd.asfreq()`を使って年の穴埋めを行います。

`pd.asfreq()`の引数には以下を設定します。  
・`freq`: 頻度を表すコード。ここでは毎年の開始日を表す`YS`を指定。  
・`fill_value`: 値の穴埋め。死亡者無しなので0とします。

それではグラフを描画します。

どんなことが読み取れますか？

#### 1-2. 文字列型からカテゴリ型への変換

今度は集計やグラフで使いやすいよう、`Nationality`をカテゴリ型に変換します。

カテゴリ数が少ない場合、カテゴリ型に変換することでデータのサイズを小さくすることができます。

まずはどんなデータが入っているか確認。

何か読み取れることがありますか？

カテゴリにするには少々数が多すぎるかもしれません。  
一方で、`Nepal`、`India`の数が極端に多いことが分かります。

そこで、今回は`Nepal`、`India`、`Others`の３つのカテゴリに分けることとします。

まずはすべてのデータをカテゴリ型に変換します。

新たに`Others`というカテゴリを追加します。

次に`Nepal`, `India`以外のデータをすべて`Others`に置き換えてしまいます。

その後、実際に変更されたことを`value_count()`で確認しましょう。

`value_counts()`で件数が0でも表示されるのは、カテゴリが存在するためです。

未使用のカテゴリを`remove_unused_categories()`で削除し、もう一度`value_count()`を実行して削除されていることを確認します。

先ほどと同じように、死亡年をX軸、死亡者数をY軸とした折れ線グラフを描いてみましょう。  
ただし、今回は`nationality_cat`ごとに線を描画します。

死亡年と`nationality_cat`でグループ化し、インデックスが`death_year`と`nationality_cat`、列が`死亡者数`のデータフレームを作成します。

先ほどと同じようにインデックスの死亡年の欠けている年をすべて埋め、死亡者数を0とします。

最後に`death_year`、`nationality_cat`をインデックスから外し、`death_year`、`nationality_cat`、`死亡者数`の３つの列となるようにします。

それでは死亡年をX軸、死亡者数をY軸とした折れ線グラフを描画します。

どんなことが読み取れますか？

### 2. 位置座標の変換

ここからはウィキペディアの「世界の山一覧 (高さ順)」にある山のリストを取得して、地図やグラフに表示してみましょう。

URL:  
[https://ja.wikipedia.org/wiki/世界の山一覧_(高さ順)](https://ja.wikipedia.org/wiki/世界の山一覧_(高さ順\))  

データの取得方法:  
`pd.read_html()`に上記URLを指定するとページ上にあるすべての`table`タグのデータがデータフレームの配列として取得できますので、そこから山のリストに該当するデータフレームを選択します。

ただし、Pythonのバージョンによってはエラーとなってデータが取得できない場合があります。
その場合は以下のGitHubのスレッドにあるように、`requests.get()`でHTMLを取得してから`pd.read_html()`を試してください。
https://github.com/pandas-dev/pandas/issues/21499


データの保存:  
取得後はセルを実行のたびに何度も取得しないよう、該当データフレームを`to_csv()`で`data/highest_montains.csv`に保存します。  
次回以降のセル実行ではファイルが存在する場合、ファイルからデータを読み込むようにしましょう。

データの中身を確認。

さっと見て、明らかにおかしい以下の二点を修正します。

1.エベレストの位置座標  
　先頭に余計なコードが含まれてしまっているようなので除去します。
 
2.存在しない列「Unnamed: 9」  
　削除します。

データの型と欠損を確認します。

順位に欠損が少々あります。

山を地図上で表示できるよう、位置座標の変換を行います。

まずはデータを確認。

フォーマットと「度・分・秒」と「度のみ」の両方の表記があることが分かりました。  
（ブラウザでは「度・分・秒」表記しか見えないのでやや肩透かしでありますが、「度のみ」の数値を抽出します。）

数値抽出のための正規表現を作り、列の中でマッチしないデータがないか確認します。

すべてにマッチするようであれば、`Series.str.extract()`で数値抽出をおこないます。

抽出した数値はそれぞれ、新たな列`latitude`、`longitude`に格納します。

### 3. 地図上にデータを表示

山を以下のルールに沿って地図上に表示してみます。
- 山の位置に円を配置。
- 円の色を高さに合わせて滑らかに色分けする。

まず滑らかな色分けのためのカラーマップを作成します。

`cm.LinearColormap`クラスのコンストラクタに任意の色の配列、山の高さの最小値、最大値を渡してインスタンスを生成します。

それでは地図上に表示します。

どんなことが読み取れますか？

地図では山の高さが認識しにくいので、棒グラフでも表示してみましょう。  
地図と見比べやすいようにX軸を経度、Yを高さとします。

8,000m峰はいくつありますか？  
また、他にどんなことが読み取れますか？

### 4. 独自の分析目標を定義して分析してみましょう

〇〇〇〇〇〇〇〇〇〇を分析。